# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GNNTrainer** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from gnn import GNNTrainer
from movie_lens_loader import MovieLensLoader
from llm import PromptBertClassifier, VanillaBertClassifier, AddingEmbeddingsBertClassifierBase

from transformers import AutoConfig

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [2]:
MODEL_NAME = "google/bert_uncased_L-2_H-128_A-2"
MODEL_HIDDEN_SIZE = AutoConfig.from_pretrained(MODEL_NAME).hidden_size
SMALL_KGE_DIMENSION = 4
LARGE_KGE_DIMENSION = MODEL_HIDDEN_SIZE
KGE_DIMENSIONS = [SMALL_KGE_DIMENSION, LARGE_KGE_DIMENSION] # Output Dimension of the GNN Encoder.
EPOCHS = 1
BATCH_SIZE = 64

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-latest-small.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [3]:

movie_lens_loader = MovieLensLoader(kge_dimensions = KGE_DIMENSIONS)

In [4]:
movie_lens_loader.llm_df.head()

,mappedUserId,mappedMovieId,title,genres,labels,prompt,split
0,0,0,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy...",1,"0[SEP]Toy Story (1995)[SEP]['Adventure', 'Anim...",val
1,0,2,Grumpier Old Men (1995),"['Comedy', 'Romance']",1,"0[SEP]Grumpier Old Men (1995)[SEP]['Comedy', '...",train
2,0,5,Heat (1995),"['Action', 'Crime', 'Thriller']",1,"0[SEP]Heat (1995)[SEP]['Action', 'Crime', 'Thr...",train
3,0,43,Seven (a.k.a. Se7en) (1995),"['Mystery', 'Thriller']",1,0[SEP]Seven (a.k.a. Se7en) (1995)[SEP]['Myster...,test
4,0,46,"Usual Suspects, The (1995)","['Crime', 'Mystery', 'Thriller']",1,"0[SEP]Usual Suspects, The (1995)[SEP]['Crime',...",train


Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
gnn_trainer =    GNNTrainer(movie_lens_loader.data, kge_dimension = SMALL_KGE_DIMENSION)
gnn_trainer_large = GNNTrainer(movie_lens_loader.data, hidden_channels=MODEL_HIDDEN_SIZE, kge_dimension=MODEL_HIDDEN_SIZE)

loading pretrained model
Device: 'cpu'
loading pretrained model
Device: 'cpu'


We then train and validate the model on the link prediction task.

If the model is already trained, we can skip this part.
Training the models can take up to 5 minutes.

In [6]:

print(gnn_trainer.kge_dimension)
gnn_trainer.train_model(movie_lens_loader.gnn_train_data, EPOCHS)
gnn_trainer.validate_model(movie_lens_loader.gnn_test_data)
print(gnn_trainer_large.kge_dimension)
gnn_trainer_large.train_model(movie_lens_loader.gnn_train_data, EPOCHS)
gnn_trainer_large.validate_model(movie_lens_loader.gnn_test_data)


4


100%|██████████| 190/190 [00:03<00:00, 49.38it/s]


Epoch: 001, Loss: 0.3376


100%|██████████| 79/79 [00:00<00:00, 127.63it/s]



Validation AUC: 0.9006
128


100%|██████████| 190/190 [00:06<00:00, 27.22it/s]


Epoch: 001, Loss: 0.3239


100%|██████████| 79/79 [00:00<00:00, 96.22it/s] 


Validation AUC: 0.9208


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

In [7]:
gnn_trainer.get_embeddings(movie_lens_loader)
gnn_trainer_large.get_embeddings(movie_lens_loader)
movie_lens_loader.llm_df.head()

,mappedUserId,mappedMovieId,title,genres,labels,prompt,split,user_embedding_4,movie_embedding_4,user_embedding_128,movie_embedding_128
0,0,0,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy...",1,"0[SEP]Toy Story (1995)[SEP]['Adventure', 'Anim...",val,"[-1.4025567770004272, 0.6816163659095764, -3.0...","[0.7183933258056641, -0.7495675086975098, -0.5...","[-0.13522014021873474, 0.16583335399627686, 0....","[-0.15604138374328613, 0.22537894546985626, 0...."
1,0,2,Grumpier Old Men (1995),"['Comedy', 'Romance']",1,"0[SEP]Grumpier Old Men (1995)[SEP]['Comedy', '...",train,"[-1.0757461786270142, 0.6430796384811401, -2.7...","[0.8827657103538513, -0.8039241433143616, -0.1...","[-0.13852912187576294, 0.22851958870887756, 0....","[-0.5181198120117188, 0.18665221333503723, 0.4..."
2,0,5,Heat (1995),"['Action', 'Crime', 'Thriller']",1,"0[SEP]Heat (1995)[SEP]['Action', 'Crime', 'Thr...",train,"[-1.1887747049331665, 0.6766993999481201, -2.8...","[0.3578329086303711, -0.37480437755584717, -0....","[-0.1273198425769806, 0.21016445755958557, 0.3...","[0.25531184673309326, 0.1213562935590744, 0.42..."
3,0,43,Seven (a.k.a. Se7en) (1995),"['Mystery', 'Thriller']",1,0[SEP]Seven (a.k.a. Se7en) (1995)[SEP]['Myster...,test,"[-1.2844634056091309, 0.6699168682098389, -2.9...","[0.49199721217155457, -0.14119231700897217, -0...","[-0.13550306856632233, 0.21558162569999695, 0....","[0.18766692280769348, 0.1714116334915161, 0.47..."
4,0,46,"Usual Suspects, The (1995)","['Crime', 'Mystery', 'Thriller']",1,"0[SEP]Usual Suspects, The (1995)[SEP]['Crime',...",train,"[-1.028777003288269, 0.6010562181472778, -2.48...","[0.6986081004142761, -0.4587530195713043, -0.2...","[-0.2011059671640396, 0.22581297159194946, 0.3...","[0.05735247954726219, 0.24271981418132782, 0.4..."


Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [8]:
vanilla_bert_classifier = VanillaBertClassifier(movie_lens_loader.llm_df, batch_size=BATCH_SIZE, model_name=MODEL_NAME)

Next we generate a vanilla llm dataset and tokenize it for training.

In [9]:
dataset_vanilla = movie_lens_loader.generate_vanilla_dataset(vanilla_bert_classifier.tokenize_function)

Next we train the model on the produced dataset. This can be skipped, if already trained ones.

In [10]:
#vanilla_bert_classifier.train_model_on_data(dataset_vanilla, epochs=EPOCHS)

Next we initialize the prompt encoder only classifier. This classifier uses the vanilla prompt and the KGEs for its link prediction.

In [11]:
prompt_bert_classifier = PromptBertClassifier(movie_lens_loader, gnn_trainer.get_embedding, kge_dimension=SMALL_KGE_DIMENSION, batch_size=BATCH_SIZE, model_max_length = 512, model_name=MODEL_NAME)

Some weights of BertForSequenceClassificationRanges were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We also generate a prompt dataset, this time the prompts also include the KGEs.

In [12]:
dataset_prompt = movie_lens_loader.generate_prompt_embedding_dataset(prompt_bert_classifier.tokenize_function, kge_dimension = prompt_bert_classifier.kge_dimension)

We also train the model. This can be skipped if already done ones.

In [13]:
#prompt_bert_classifier.train_model_on_data(dataset_prompt, epochs = EPOCHS)

In [14]:
adding_embedding_bert_only_classifier = AddingEmbeddingsBertClassifierBase(movie_lens_loader, gnn_trainer_large.get_embedding, kge_dimension=MODEL_HIDDEN_SIZE, batch_size=BATCH_SIZE, model_name=MODEL_NAME)
dataset_adding_embedding = movie_lens_loader.generate_adding_embedding_dataset(adding_embedding_bert_only_classifier.tokenizer.sep_token, adding_embedding_bert_only_classifier.tokenizer.pad_token, adding_embedding_bert_only_classifier.tokenize_function, kge_dimension = MODEL_HIDDEN_SIZE)

Some weights of AddingEmbeddingBertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/56469 [00:00<?, ? examples/s]

Map:   0%|          | 0/17142 [00:00<?, ? examples/s]

Map:   0%|          | 0/17142 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/56469 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/17142 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/17142 [00:00<?, ? examples/s]

In [15]:
#adding_embedding_bert_only_classifier.train_model_on_data(dataset_adding_embedding, epochs = EPOCHS)

  0%|          | 0/883 [00:00<?, ?it/s]

: 

In [ ]:
movie_lens_loader.llm_df["labels"] = 1
movie_lens_loader.add_false_edges(false_ratio = 2.0)

Adding 656 false edges for train.
Adding 196 false edges for test.
Adding 172 false edges for val.


,mappedUserId,mappedMovieId,title,genres,prompt,split,labels,user_embedding_4,movie_embedding_4,user_embedding_128,movie_embedding_128
0,180,910,Once Upon a Time in the West (C'era una volta ...,"['Action', 'Drama', 'Western']",180[SEP]Once Upon a Time in the West (C'era un...,train,1,"[0.838445246219635, -0.37525758147239685, -2.3...","[1.3539336919784546, -1.3562633991241455, -0.4...","[0.040662720799446106, 0.06649568676948547, -0...","[-0.4420027732849121, -0.9158413410186768, 0.3..."
1,83,5738,"Aviator, The (2004)",['Drama'],"83[SEP]Aviator, The (2004)[SEP]['Drama']",train,1,"[-0.5373823046684265, 0.5054466724395752, -3.0...","[-0.1709502637386322, -1.4477055072784424, -1....","[-0.41327977180480957, -0.916988730430603, -0....","[-0.3103116750717163, 0.22894519567489624, -0...."
2,260,656,Escape from L.A. (1996),"['Action', 'Adventure', 'Sci-Fi', 'Thriller']","260[SEP]Escape from L.A. (1996)[SEP]['Action',...",train,1,"[-0.9165530800819397, 1.343902349472046, -2.71...","[1.0745933055877686, -0.7053966522216797, -0.6...","[0.09863702952861786, 0.3924756944179535, 0.99...","[0.04699868708848953, -0.07751499116420746, 0...."
3,355,1576,Indiana Jones and the Temple of Doom (1984),"['Action', 'Adventure', 'Fantasy']",355[SEP]Indiana Jones and the Temple of Doom (...,train,1,"[0.834461510181427, 0.3378297686576843, -2.922...","[0.6744173765182495, -0.8369619846343994, -1.1...","[-0.06641509383916855, -0.24213919043540955, -...","[-0.00337088480591774, -0.00557747483253479, 0..."
4,229,4643,Gloomy Sunday (Ein Lied von Liebe und Tod) (1999),"['Drama', 'Romance']",229[SEP]Gloomy Sunday (Ein Lied von Liebe und ...,train,1,"[0.018658041954040527, -0.3827669322490692, -3...","[0.11454576253890991, -2.884582996368408, 0.77...","[-0.010033026337623596, -0.3314223289489746, 0...","[-0.6175022125244141, 0.7383372783660889, -1.0..."
...,...,...,...,...,...,...,...,...,...,...,...
1531,131,1730,Life Is Beautiful (La Vita è bella) (1997),"['Comedy', 'Drama', 'Romance', 'War']",131[SEP]Life Is Beautiful (La Vita è bella) (1...,val,0,"[0.6431354284286499, -0.11565425246953964, -2....","[0.3669813871383667, -0.5662603378295898, -1.3...","[0.21160167455673218, -0.4200924336910248, -0....","[-0.26695168018341064, 0.08760415017604828, -0..."
1532,310,3898,Halloween: Resurrection (Halloween 8) (2002),"['Horror', 'Thriller']",310[SEP]Halloween: Resurrection (Halloween 8) ...,val,0,"[0.19500422477722168, -0.1430656909942627, -3....","[0.6304952502250671, -1.9827512502670288, 0.07...","[0.06075066328048706, -0.44455862045288086, -0...","[0.12356743216514587, -0.02117633819580078, 0...."
1533,566,1063,Grease (1978),"['Comedy', 'Musical', 'Romance']","566[SEP]Grease (1978)[SEP]['Comedy', 'Musical'...",val,0,"[0.24057187139987946, 0.04831351339817047, -3....","[1.3200078010559082, -0.9709514379501343, -0.7...","[0.005273051559925079, 0.0785430520772934, -0....","[-0.4090748727321625, 0.06467289477586746, 0.1..."
1534,532,1073,Jerry Maguire (1996),"['Drama', 'Romance']","532[SEP]Jerry Maguire (1996)[SEP]['Drama', 'Ro...",val,0,"[0.15030480921268463, 0.42936304211616516, -2....","[1.0465419292449951, -0.7236583232879639, -0.8...","[-0.06371378898620605, -0.9162569642066956, 0....","[-0.43918490409851074, 0.23587584495544434, -0..."


In [ ]:
vanilla_bert_classifier = VanillaBertClassifier(movie_lens_loader.llm_df, batch_size=BATCH_SIZE, model_name=MODEL_NAME, false_ratio=-1)
prompt_bert_classifier = PromptBertClassifier(movie_lens_loader, gnn_trainer.get_embedding, kge_dimension=SMALL_KGE_DIMENSION, batch_size=BATCH_SIZE, model_max_length = 512, model_name=MODEL_NAME, false_ratio = -1)
adding_embedding_bert_only_classifier = AddingEmbeddingsBertClassifierBase(movie_lens_loader, gnn_trainer_large.get_embedding, kge_dimension=MODEL_HIDDEN_SIZE, batch_size=BATCH_SIZE, model_name=MODEL_NAME, false_ratio=-1)


In [ ]:
#del dataset_vanilla
#del dataset_prompt
#del dataset_adding_embedding
#import gc
#gc.collect()
dataset_vanilla = movie_lens_loader.generate_vanilla_dataset(vanilla_bert_classifier.tokenize_function, suffix = "_fixed", force_recompute=True)
dataset_prompt = movie_lens_loader.generate_prompt_embedding_dataset(prompt_bert_classifier.tokenize_function, kge_dimension = prompt_bert_classifier.kge_dimension, suffix = "_fixed", force_recompute=True)
dataset_adding_embedding = movie_lens_loader.generate_adding_embedding_dataset(adding_embedding_bert_only_classifier.tokenizer.sep_token, adding_embedding_bert_only_classifier.tokenizer.pad_token, adding_embedding_bert_only_classifier.tokenize_function, suffix = "_fixed", kge_dimension = MODEL_HIDDEN_SIZE, force_recompute=True)

Map:   0%|          | 0/984 [00:00<?, ? examples/s]

Map:   0%|          | 0/258 [00:00<?, ? examples/s]

Map:   0%|          | 0/294 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/984 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/258 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/258 [00:00<?, ? examples/s]

Map:   0%|          | 0/294 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/984 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/258 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/294 [00:00<?, ? examples/s]

In [ ]:
vanilla_bert_classifier.forward_dataset_and_save_outputs(dataset_vanilla, force_recompute=True)
prompt_bert_classifier.forward_dataset_and_save_outputs(dataset_prompt, force_recompute=True)
df = adding_embedding_bert_only_classifier.forward_dataset_and_save_outputs(dataset_adding_embedding, force_recompute=True)

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Vanilla train Forward Epoch 1 from 3


ValueError: invalid literal for int() with base 10: ''